In [ ]:
# Poisson problem on a unit square with Dirichlet and natural boundary
# conditions. Manufactured solution.

import numpy as np
import eztfem as ezt
from func import func
from scipy.sparse.linalg import spsolve


In [ ]:
# create mesh
mesh = ezt.quadrilateral2d([20, 20], 'quad9', length=np.array([1.2, 1]))


In [ ]:
# define the problem
elementdof = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
                       [2, 2, 2, 2, 2, 2, 2, 2, 2]], dtype=int).transpose()
problem = ezt.Problem(mesh, elementdof, nphysq=1)


In [ ]:
# define Gauss integration and basis functions
user = ezt.User()
shape = 'quad'
user.xr, user.wg = ezt.gauss_legendre(shape, n=3)
user.phi, user.dphi = ezt.basis_function(shape, 'Q2', user.xr)


In [ ]:
# user struct for setting problem coefficients, ...
user.coorsys = 0
user.alpha = 1
user.funcnr = 7
user.func = func


In [ ]:
# assemble the system matrix and vector
A, f = ezt.build_system(mesh, problem, ezt.poisson_elem, user)


In [ ]:
# define Gauss integration and basis functions (for boundary integral)
xr_line, user.wg = ezt.gauss_legendre('line', n=3)
user.phi, user.dphi = ezt.basis_function('line', 'P2', xr_line)


In [ ]:
# add natural boundary condition
user.funcnr = 8
ezt.add_boundary_elements(mesh, problem, f, ezt.poisson_natboun_curve,
                              user, curve=1)


In [ ]:
# define essential boundary conditions (Dirichlet)
iess = ezt.define_essential(mesh, problem, 'curves', [0, 2, 3])


In [ ]:
# fill values for the essential boundary conditions
uess = ezt.fill_system_vector(mesh, problem, 'curves', [0, 2, 3],
                                  func, funcnr=6)


In [ ]:
# apply essential boundary conditions to the system
ezt.apply_essential(A, f, uess, iess)


In [ ]:
# solve the system
u = spsolve(A.tocsr(), f)


In [ ]:
# compare with exact solution
print('Difference with exact solution:')
uex = ezt.fill_system_vector(mesh, problem, 'nodes',
                                 np.arange(mesh.nnodes), func, funcnr=6)
maxdiff = np.max(np.abs(u - uex))
print(maxdiff)


In [ ]:
# gradient (dudx,dudy) of the solution
xr = ezt.refcoor_nodal_points(mesh)
user.phi, user.dphi = ezt.basis_function('quad', 'Q2', xr)
user.u = u
gradu = ezt.deriv_vector(mesh, problem, ezt.poisson_deriv, user)


In [ ]:
# create a PyVista mesh for visualizing results
mesh_pv = ezt.generate_pyvista_mesh(mesh)


In [ ]:
# plot the solution field
ezt.plot_sol(mesh_pv, problem, u, show_scalar_bar=True, n_colors=16,
             window_size=(800, 400))
